# MLB Digital Engagement Data Analysis

In [1]:
import pandas as pd
import numpy as np
import re
import sys
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pickle
pd.set_option('display.max_columns', None)

In [39]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

## Data Load

In [3]:
players = pd.read_pickle('players.pickle')
players

,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,665482,Gilberto Celestino,1999-02-13,2021-06-02,Santo Domingo,NaN,Dominican Republic,72,170,8,Outfielder,False
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True
2,661269,Vladimir Gutierrez,1995-09-18,2021-05-28,Havana,NaN,Cuba,73,190,1,Pitcher,True
3,669212,Eli Morgan,1996-05-13,2021-05-28,Rancho Palos Verdes,CA,USA,70,190,1,Pitcher,True
4,666201,Alek Manoah,1998-01-09,2021-05-27,Homestead,FL,USA,78,260,1,Pitcher,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2056,672695,Geraldo Perdomo,1999-10-22,NaN,Santo Domingo,NaN,Dominican Republic,74,203,6,Shortstop,True
2057,672911,Jesus Cruz,1995-04-15,NaN,Salinas de Hidalgo,San Luis Potosi,Mexico,73,230,1,Pitcher,False
2058,676103,Damon Jones,1994-09-30,NaN,Twin Falls,ID,USA,77,233,1,Pitcher,True
2059,676755,Isaac Mattson,1995-07-14,NaN,Erie,PA,USA,74,205,1,Pitcher,True


In [4]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   playerId                        2061 non-null   int64 
 1   playerName                      2061 non-null   object
 2   DOB                             2061 non-null   object
 3   mlbDebutDate                    2025 non-null   object
 4   birthCity                       2061 non-null   object
 5   birthStateProvince              1516 non-null   object
 6   birthCountry                    2061 non-null   object
 7   heightInches                    2061 non-null   int16 
 8   weight                          2061 non-null   int32 
 9   primaryPositionCode             2061 non-null   object
 10  primaryPositionName             2061 non-null   object
 11  playerForTestSetAndFuturePreds  2057 non-null   object
dtypes: int16(1), int32(1), int64(1), object(9)
memor

In [5]:
playersfinal = players[(players.playerForTestSetAndFuturePreds ==True) & (players.primaryPositionName != 'Pitcher')]
playersfinal

,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True
5,680911,Owen Miller,1996-11-15,2021-05-23,Mequon,WI,USA,72,185,4,Second Base,True
6,670764,Taylor Walls,1996-07-10,2021-05-22,Americus,GA,USA,70,185,6,Shortstop,True
7,608422,Jose Godoy,1994-10-13,2021-05-21,Maracaibo,NaN,Venezuela,71,200,2,Catcher,True
9,642456,Luis Barrera,1995-11-15,2021-05-19,Tamboril,NaN,Dominican Republic,72,195,8,Outfielder,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2036,656180,Riley Adams,1996-06-26,NaN,Encinitas,CA,USA,76,246,2,Catcher,True
2050,666128,Mario Feliciano,1998-11-20,NaN,Bayamon,NaN,Puerto Rico,73,200,2,Catcher,True
2054,666801,Rodolfo Castro,1999-05-21,NaN,Los Llanos,NaN,Dominican Republic,72,205,4,Second Base,True
2055,667674,Jack Kruger,1994-10-26,NaN,Los Angeles,CA,USA,73,195,2,Catcher,True


In [6]:
roster = pd.read_pickle('train_rosters.pickle')
roster

,dailyDataDate,playerId,gameDate,teamId,statusCode,status
0,20180101,400121,2018-01-01,116,A,Active
1,20180101,408045,2018-01-01,142,A,Active
2,20180101,425492,2018-01-01,120,A,Active
3,20180101,429664,2018-01-01,136,A,Active
4,20180101,431151,2018-01-01,121,A,Active
...,...,...,...,...,...,...
1489655,20210430,676969,2021-04-30,140,A,Active
1489656,20210430,677960,2021-04-30,135,A,Active
1489657,20210430,680692,2021-04-30,145,RM,Reassigned to Minors
1489658,20210430,680702,2021-04-30,136,RM,Reassigned to Minors


In [7]:
teams = pd.read_pickle('teams.pickle')
teams

,id,name,teamName,teamCode,shortName,abbreviation,locationName,leagueId,leagueName,divisionId,divisionName,venueId,venueName
0,108,Los Angeles Angels,Angels,ana,LA Angels,LAA,Anaheim,103,American League,200,American League West,1,Angel Stadium
1,109,Arizona Diamondbacks,D-backs,ari,Arizona,ARI,Phoenix,104,National League,203,National League West,15,Chase Field
2,110,Baltimore Orioles,Orioles,bal,Baltimore,BAL,Baltimore,103,American League,201,American League East,2,Oriole Park at Camden Yards
3,111,Boston Red Sox,Red Sox,bos,Boston,BOS,Boston,103,American League,201,American League East,3,Fenway Park
4,112,Chicago Cubs,Cubs,chn,Chi Cubs,CHC,Chicago,104,National League,205,National League Central,17,Wrigley Field
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,144,Atlanta Braves,Braves,atl,Atlanta,ATL,Atlanta,104,National League,204,National League East,4705,Truist Park
26,145,Chicago White Sox,White Sox,cha,Chi White Sox,CWS,Chicago,103,American League,202,American League Central,4,Guaranteed Rate Field
27,146,Miami Marlins,Marlins,mia,Miami,MIA,Miami,104,National League,204,National League East,4169,loanDepot park
28,147,New York Yankees,Yankees,nya,NY Yankees,NYY,Bronx,103,American League,201,American League East,3313,Yankee Stadium


In [8]:
teams['abbreviation'].unique()

array(['LAA', 'ARI', 'BAL', 'BOS', 'CHC', 'CIN', 'CLE', 'COL', 'DET',
       'HOU', 'KC', 'LAD', 'WSH', 'NYM', 'OAK', 'PIT', 'SD', 'SEA', 'SF',
       'STL', 'TB', 'TEX', 'TOR', 'MIN', 'PHI', 'ATL', 'CWS', 'MIA',
       'NYY', 'MIL'], dtype=object)

In [9]:
def patterns(i):
    pattern1 = re.compile(r'\w*LAA\w*')
    pattern2 = re.compile(r'\w*BOS\w*')
    pattern3 = re.compile(r'\w*ATL\w*')
    pattern4 = re.compile(r'\w*CWS\w*')
    pattern5 = re.compile(r'\w*NYY\w*')
    pattern6 = re.compile(r'\w*NYM\w*')
    pattern7 = re.compile(r'\w*LAD\w*')
    pattern8 = re.compile(r'\w*WSH\w*')
    pattern9 = re.compile(r'\w*TEX\w*')
    pattern10 = re.compile(r'\w*CHC\w*')
    pattern11 = re.compile(r'\w*PHI\w*')
    
    if pattern1.search(i) != None:
        return 1
    elif pattern2.search(i) != None:
        return 1
    elif pattern3.search(i) != None:
        return 1
    elif pattern4.search(i) != None:
        return 1
    elif pattern5.search(i) != None:
        return 1
    elif pattern6.search(i) != None:
        return 1
    elif pattern7.search(i) != None:
        return 1
    elif pattern8.search(i) != None:
        return 1
    elif pattern9.search(i) != None:
        return 1
    elif pattern10.search(i) != None:
        return 1
    elif pattern11.search(i) != None:
        return 1
    else:
        return 0

In [10]:
teams['majormarket'] = teams['abbreviation'].apply(patterns)

In [11]:
teams

,id,name,teamName,teamCode,shortName,abbreviation,locationName,leagueId,leagueName,divisionId,divisionName,venueId,venueName,majormarket
0,108,Los Angeles Angels,Angels,ana,LA Angels,LAA,Anaheim,103,American League,200,American League West,1,Angel Stadium,1
1,109,Arizona Diamondbacks,D-backs,ari,Arizona,ARI,Phoenix,104,National League,203,National League West,15,Chase Field,0
2,110,Baltimore Orioles,Orioles,bal,Baltimore,BAL,Baltimore,103,American League,201,American League East,2,Oriole Park at Camden Yards,0
3,111,Boston Red Sox,Red Sox,bos,Boston,BOS,Boston,103,American League,201,American League East,3,Fenway Park,1
4,112,Chicago Cubs,Cubs,chn,Chi Cubs,CHC,Chicago,104,National League,205,National League Central,17,Wrigley Field,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,144,Atlanta Braves,Braves,atl,Atlanta,ATL,Atlanta,104,National League,204,National League East,4705,Truist Park,1
26,145,Chicago White Sox,White Sox,cha,Chi White Sox,CWS,Chicago,103,American League,202,American League Central,4,Guaranteed Rate Field,1
27,146,Miami Marlins,Marlins,mia,Miami,MIA,Miami,104,National League,204,National League East,4169,loanDepot park,0
28,147,New York Yankees,Yankees,nya,NY Yankees,NYY,Bronx,103,American League,201,American League East,3313,Yankee Stadium,1


In [12]:
seasons = pd.read_pickle('seasons.pickle')
seasons

,seasonId,seasonStartDate,seasonEndDate,preSeasonStartDate,preSeasonEndDate,regularSeasonStartDate,regularSeasonEndDate,lastDate1stHalf,allStarDate,firstDate2ndHalf,postSeasonStartDate,postSeasonEndDate
0,2017,2017-04-02,2017-11-01,2017-02-22,2017-04-01,2017-04-02,2017-10-01,2017-07-09,2017-07-11,2017-07-14,2017-10-03,2017-11-01
1,2018,2018-03-29,2018-10-28,2018-02-21,2018-03-27,2018-03-29,2018-10-01,2018-07-15,2018-07-17,2018-07-19,2018-10-02,2018-10-28
2,2019,2019-03-20,2019-10-30,2019-02-21,2019-03-26,2019-03-20,2019-09-29,2019-07-07,2019-07-09,2019-07-11,2019-10-01,2019-10-30
3,2020,2020-07-23,2020-10-28,2020-02-21,2020-07-22,2020-07-23,2020-09-27,2020-08-25,NaN,2020-08-26,2020-09-29,2020-10-28
4,2021,2021-02-28,2021-10-31,2021-02-28,2021-03-30,2021-04-01,2021-10-03,2021-07-11,2021-07-13,2021-07-15,2021-10-04,2021-10-31


In [13]:
events = pd.read_pickle('train_events.pickle')
events

,dailyDataDate,gamePk,gameDate,gameTimeUTC,season,gameType,playId,eventId,inning,halfInning,homeScore,awayScore,menOnBase,atBatIndex,atBatDesc,atBatEvent,hasOut,pitcherTeamId,isPitcherHome,pitcherTeam,hitterTeamId,hitterTeam,pitcherId,pitcherName,isStarter,pitcherHand,hitterId,hitterName,batSide,pitchNumber,balls,strikes,isGB,isLD,isFB,isPU,launchSpeed,launchAngle,totalDistance,event,description,rbi,pitchType,call,outs,inPlay,isPaOver,startSpeed,endSpeed,nastyFactor,breakAngle,breakLength,breakY,spinRate,spinDirection,pX,pZ,aX,aY,aZ,pfxX,pfxZ,vX0,vY0,vZ0,x,y,x0,y0,z0,type,zone
0,20180329,529409,2018-03-29,2018-03-30T02:10:00Z,2018,R,None,5,9,top,2,1,None,61,Lonnie Chisenhall hit by pitch.,Hit By Pitch,0,136,1,Seattle Mariners,114,Cleveland Indians,621242,Edwin Diaz,0,R,502082,Lonnie Chisenhall,L,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Balk,"With Lonnie Chisenhall batting, Rajai Davis advances to 2nd on a balk.",NaN,None,None,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,action,NaN
1,20180329,529416,2018-03-29,2018-03-29T20:10:00Z,2018,R,None,0,3,bottom,0,1,None,18,Ryan Flaherty singles on a bunt ground ball to third baseman Maikel Franco.,Single,0,143,0,Philadelphia Phillies,144,Atlanta Braves,605400,Aaron Nola,1,R,475247,Ryan Flaherty,L,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Injury,Atlanta Braves catcher Tyler Flowers left the game due to an injured side.,NaN,None,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,action,NaN
2,20180329,529416,2018-03-29,2018-03-29T20:10:00Z,2018,R,None,4,9,top,5,5,None,72,Carlos Santana strikes out swinging.,Strikeout,1,144,1,Atlanta Braves,143,Philadelphia Phillies,527055,Arodys Vizcaino,0,R,467793,Carlos Santana,L,5,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ejection,Atlanta Braves Manager Brian Snitker ejected by 3B umpire Jordan Baker.,NaN,None,None,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,action,NaN
3,20180329,529411,2018-03-29,2018-03-29T19:35:00Z,2018,R,None,4,3,bottom,0,2,None,23,Delino DeShields strikes out swinging.,Strikeout,1,117,0,Houston Astros,140,Texas Rangers,434378,Justin Verlander,1,R,592261,Delino DeShields,R,3,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pickoff 1B,"With Delino DeShields batting, Justin Verlander picks off Rougned Odor at 1st on throw to Marwin Gonzalez.",NaN,None,None,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,action,NaN
4,20180329,529406,2018-03-29,2018-03-29T20:00:00Z,2018,R,None,5,8,top,0,4,None,54,Hanley Ramirez walks.,Walk,1,139,1,Tampa Bay Rays,111,Boston Red Sox,643493,Austin Pruitt,0,R,434670,Hanley Ramirez,R,5,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Runner Out,"Mookie Betts out at 1st, catcher Wilson Ramos to first baseman C. J. Cron.",NaN,None,None,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,action,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017649,20210430,634327,2021-04-30,2021-04-30T23:05:00Z,2021,R,fbf14c5b-6e3e-4ccc-b8f9-98a41f81fd1d,6,1,bottom,0,0,Men_On,6,Gio Urshela singles on a line drive to left fielder Harold Castro. Giancarlo Stanton scores.,Single,0,116,0,Detroit Tigers,147,New York Yankees,669373,Tarik Skubal,1,L,570482,Gio Urshela,R,6,2,2,0.0,0.0,0.0,0.0,NaN,NaN,NaN,None,None,1.0,FF,F,2,0.0,0.0,95.699997,88.699997,NaN,24.0,2.4,24.0,2400.0,152.0,-0.01,1.81,7.48,26.990000,-11.19,3.77,10.58,-8.45,-138.960007,-9.82,117.559998,189.869995,2.56,50.000000,6.10,pitch,8.0
2017650,20210430,634328,2021-04-30,2021-05-01T01:40:00Z,2021,R,def99581-1dde-40fd-9d7a-5811f6f90df5,2,8,top,5,2,Empty,63,Charlie Blackmon lines out to left fielder David Peralta.,L

In [14]:
games = pd.read_pickle('train_games.pickle')
games

,dailyDataDate,gamePk,gameType,season,gameDate,gameTimeUTC,resumeDate,resumedFrom,codedGameState,detailedGameState,isTie,gameNumber,doubleHeader,dayNight,scheduledInnings,gamesInSeries,seriesDescription,homeId,homeName,homeAbbrev,homeWins,homeLosses,homeWinPct,homeWinner,homeScore,awayId,awayName,awayAbbrev,awayWins,awayLosses,awayWinPct,awayWinner,awayScore
0,20180221,533782,E,2018,2018-02-21,2018-02-21T20:10:00Z,NaN,NaN,F,Final,0.0,1,N,day,7,0.0,Exhibition,109,Arizona Diamondbacks,ARI,1,0,1.000,True,7.0,5035,Arizona State Sun Devils,ASU,0.0,1.0,0.000,False,2.0
1,20180222,534461,E,2018,2018-02-22,2018-02-22T18:05:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,0.0,Exhibition,116,Detroit Tigers,DET,1,0,1.000,True,6.0,228,Florida Southern College Mocs,FSC,0.0,1.0,0.000,False,1.0
2,20180222,545334,E,2018,2018-02-22,2018-02-22T18:05:00Z,NaN,NaN,F,Final,0.0,1,N,day,9,0.0,Exhibition,143,Philadelphia Phillies,PHI,1,0,1.000,True,6.0,231,University of Tampa Spartans,UT,0.0,1.0,0.000,False,0.0
3,20180222,547295,E,2018,2018-02-22,2018-02-22T03:33:00Z,NaN,NaN,F,Final,0.0,2,Y,night,9,0.0,Exhibition,111,Boston Red Sox,BOS,2,0,1.000,True,4.0,227,Boston College Eagles,BC,0.0,1.0,0.000,False,2.0
4,20180222,533784,E,2018,2018-02-22,2018-02-22T23:05:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,0.0,Exhibition,142,Minnesota Twins,MIN,1,0,1.000,True,2.0,4864,Minnesota Gophers,UM,0.0,1.0,0.000,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7855,20210430,634275,R,2021,2021-04-30,2021-04-30T23:10:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,3.0,Regular Season,139,Tampa Bay Rays,TB,13,14,0.481,False,2.0,117,Houston Astros,HOU,14.0,12.0,0.538,True,9.0
7856,20210430,634391,R,2021,2021-04-30,2021-05-01T00:10:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,3.0,Regular Season,142,Minnesota Twins,MIN,9,15,0.375,True,9.0,118,Kansas City Royals,KC,15.0,9.0,0.625,False,1.0
7857,20210430,634305,R,2021,2021-04-30,2021-04-30T23:10:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,3.0,Regular Season,113,Cincinnati Reds,CIN,12,13,0.480,True,8.0,112,Chicago Cubs,CHC,11.0,15.0,0.423,False,6.0
7858,20210430,634280,R,2021,2021-04-30,2021-04-30T23:37:00Z,NaN,NaN,F,Final,0.0,1,N,night,9,3.0,Regular Season,141,Toronto Blue Jays,TOR,12,12,0.500,True,13.0,144,Atlanta Braves,ATL,12.0,14.0,0.462,False,5.0


In [15]:
playerBoxScores = pd.read_pickle('train_playerBoxScores.pickle')
playerBoxScores

,dailyDataDate,home,gamePk,gameDate,gameTimeUTC,teamId,teamName,playerId,playerName,jerseyNum,positionCode,positionName,positionType,battingOrder,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,winsPitching,lossesPitching,flyOutsPitching,airOutsPitching,groundOutsPitching,runsPitching,doublesPitching,triplesPitching,homeRunsPitching,strikeOutsPitching,baseOnBallsPitching,intentionalWalksPitching,hitsPitching,hitByPitchPitching,atBatsPitching,caughtStealingPitching,stolenBasesPitching,inningsPitched,saveOpportunities,earnedRuns,battersFaced,outsPitching,pitchesThrown,balls,strikes,hitBatsmen,balks,wildPitches,pickoffsPitching,rbiPitching,gamesFinishedPitching,inheritedRunners,inheritedRunnersScored,catchersInterferencePitching,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances
0,20180329,1,529418,2018-03-29,2018-03-29T23:08:00Z,119,Los Angeles Dodgers,605131,Austin Barnes,15,12,Pinch Runner,Runner,601.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20180329,1,529406,2018-03-29,2018-03-29T20:00:00Z,139,Tampa Bay Rays,605480,Mallex Smith,0,7,Outfielder,Outfielder,601.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2,20180329,0,529416,2018-03-29,2018-03-29T20:10:00Z,143,Philadelphia Phillies,546318,Odubel Herrera,37,8,Outfielder,Outfielder,401.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,20180329,0,529412,2018-03-29,2018-03-29T20:05:00Z,108,Los Angeles Angels,527043,Jefry Marte,19,3,First Base,Infielder,402.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
4,20180329,1,529408,2018-03-29,2018-03-29T20:15:00Z,118,Kansas City Royals,449181,Paulo Orlando,16,8,Outfielder,Outfielder,701.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185139,20210430,1,634303,2021-04-30,2021-05-01T02:10:00Z,135,San Diego Padres,506433,Yu Darvish,11,1,Pitcher,Pitcher,900.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,4.0,4.0,3.0,1.0,2.0,0.0,1.0,12.0,3.0,0.0,4.0,1.0,23.0,0.0,0.0,6.1,0.0,1.0,27.0,19.0,107.0,38.0,69.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [16]:
nextDayPlayerEngagement = pd.read_pickle('train_nextDayPlayerEngagement.pickle')
nextDayPlayerEngagement

,dailyDataDate,engagementMetricsDate,playerId,target1,target2,target3,target4
0,20180101,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294
1,20180101,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118
2,20180101,2018-01-02,519317,0.974327,56.177044,13.693746,64.166664
3,20180101,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745
4,20180101,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373
...,...,...,...,...,...,...,...
2506171,20210430,2021-05-01,451661,0.000000,0.013314,0.000000,0.625925
2506172,20210430,2021-05-01,519301,0.000131,0.003329,0.000000,0.216229
2506173,20210430,2021-05-01,527055,0.000000,0.019971,0.000000,0.273131
2506174,20210430,2021-05-01,543484,0.000131,0.056586,0.000000,1.024240


In [17]:
awards = pd.read_pickle('train_awards.pickle')
awards

,dailyDataDate,awardId,awardName,awardDate,awardSeason,playerId,playerName,awardPlayerTeamId
0,20180115,VWLCPOY,VWL Comeback Player of the Year,2018-01-15,2017,150119,Freddy Garcia,699.0
1,20180115,VWLPOY,VWL Pitcher of the Year,2018-01-15,2017,446861,Guillermo Moscoso,699.0
2,20180115,VWLMOY,VWL Manager of the Year,2018-01-15,2017,492527,Mike Rojas,695.0
3,20180115,VWLRLOY,VWL Reliever of the Year,2018-01-15,2017,519246,Andres Santiago,694.0
4,20180115,VWLOPOY,VWL Offensive Player of the Year,2018-01-15,2017,543874,Jose Vargas,699.0
...,...,...,...,...,...,...,...,...
5910,20210430,NLPOM,NL Player of the Month,2021-04-30,2021,660670,Ronald Acuna Jr.,144.0
5911,20210430,ALRRELMON,AL Reliever of the Month,2021-04-30,2021,598264,Matt Barnes,111.0
5912,20210430,ALPOM,AL Player of the Month,2021-04-30,2021,621439,Byron Buxton,142.0
5913,20210430,ALROM,AL Rookie of the Month,2021-04-30,2021,606213,Yermin Mercedes,145.0


In [18]:
playerTwitterFollowers = pd.read_pickle('train_playerTwitterFollowers.pickle')
playerTwitterFollowers

,dailyDataDate,date,playerId,playerName,accountName,twitterHandle,numberOfFollowers
0,20180101,2018-01-01,545361,Mike Trout,Mike Trout,@miketrout,2452409
1,20180101,2018-01-01,506433,Yu Darvish,Yu Darvish,@faridyu,1945081
2,20180101,2018-01-01,434378,Justin Verlander,Justin Verlander,@justinverlander,1795985
3,20180101,2018-01-01,430897,Nick Swisher,Nick Swisher,@nickswisher,1711807
4,20180101,2018-01-01,120074,David Ortiz,David Ortiz,@davidortiz,1515463
...,...,...,...,...,...,...,...
39584,20210401,2021-04-01,605125,Cody Asche,Cody Asche,@aschecody,11
39585,20210401,2021-04-01,542953,Buddy Boshers,Buddy Boshers,@buddy4023,9
39586,20210401,2021-04-01,425766,James Loney,James Loney,@james_loney,5
39587,20210401,2021-04-01,605233,Derek Fisher,Derek Fisher,@defish23,5


## Join Data Sets of Interest

In [19]:
df1 = pd.merge(playersfinal, playerBoxScores, how='left', left_on=['playerId'], right_on=['playerId'])
df1

,playerId,playerName_x,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds,dailyDataDate,home,gamePk,gameDate,gameTimeUTC,teamId,teamName,playerName_y,jerseyNum,positionCode,positionName,positionType,battingOrder,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,winsPitching,lossesPitching,flyOutsPitching,airOutsPitching,groundOutsPitching,runsPitching,doublesPitching,triplesPitching,homeRunsPitching,strikeOutsPitching,baseOnBallsPitching,intentionalWalksPitching,hitsPitching,hitByPitchPitching,atBatsPitching,caughtStealingPitching,stolenBasesPitching,inningsPitched,saveOpportunities,earnedRuns,battersFaced,outsPitching,pitchesThrown,balls,strikes,hitBatsmen,balks,wildPitches,pickoffsPitching,rbiPitching,gamesFinishedPitching,inheritedRunners,inheritedRunnersScored,catchersInterferencePitching,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances
0,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,680911,Owen Miller,1996-11-15,2021-05-23,Mequon,WI,USA,72,185,4,Second Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,670764,Taylor Walls,1996-07-10,2021-05-22,Americus,GA,USA,70,185,6,Shortstop,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,608422,Jose Godoy,1994-10-13,2021-05-21,Maracaibo,NaN,Venezuela,71,200,2,Catcher,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,642456,Luis Barrera,1995-11-15,2021-05-19,Tamboril,NaN,Dominican Republic,72,195,8,Outfielder,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100184,667674,Jack Kruger,1994-10-26,NaN,Los Angeles,CA,USA,73,195,2,Catcher,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [20]:
df2 = pd.merge(df1, teams, how='left', left_on=['teamId'], right_on=['id'])
df2

,playerId,playerName_x,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds,dailyDataDate,home,gamePk,gameDate,gameTimeUTC,teamId,teamName_x,playerName_y,jerseyNum,positionCode,positionName,positionType,battingOrder,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,winsPitching,lossesPitching,flyOutsPitching,airOutsPitching,groundOutsPitching,runsPitching,doublesPitching,triplesPitching,homeRunsPitching,strikeOutsPitching,baseOnBallsPitching,intentionalWalksPitching,hitsPitching,hitByPitchPitching,atBatsPitching,caughtStealingPitching,stolenBasesPitching,inningsPitched,saveOpportunities,earnedRuns,battersFaced,outsPitching,pitchesThrown,balls,strikes,hitBatsmen,balks,wildPitches,pickoffsPitching,rbiPitching,gamesFinishedPitching,inheritedRunners,inheritedRunnersScored,catchersInterferencePitching,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances,id,name,teamName_y,teamCode,shortName,abbreviation,locationName,leagueId,leagueName,divisionId,divisionName,venueId,venueName,majormarket
0,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,680911,Owen Miller,1996-11-15,2021-05-23,Mequon,WI,USA,72,185,4,Second Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,670764,Taylor Walls,1996-07-10,2021-05-22,Americus,GA,USA,70,185,6,Shortstop,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,608422,Jose Godoy,1994-10-13,2021-05-21,Maracaibo,NaN,Venezuela,71,200,2,Catcher,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,642456,Luis Barrera,1995-11-15,2021-05-19,Tamboril,NaN,Dominican Republic,72,195,8,Outfielder,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [21]:
df3 = pd.merge(df2, awards, how='left', left_on=['playerId','dailyDataDate'], right_on=['playerId','dailyDataDate'])
df3

,playerId,playerName_x,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds,dailyDataDate,home,gamePk,gameDate,gameTimeUTC,teamId,teamName_x,playerName_y,jerseyNum,positionCode,positionName,positionType,battingOrder,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,winsPitching,lossesPitching,flyOutsPitching,airOutsPitching,groundOutsPitching,runsPitching,doublesPitching,triplesPitching,homeRunsPitching,strikeOutsPitching,baseOnBallsPitching,intentionalWalksPitching,hitsPitching,hitByPitchPitching,atBatsPitching,caughtStealingPitching,stolenBasesPitching,inningsPitched,saveOpportunities,earnedRuns,battersFaced,outsPitching,pitchesThrown,balls,strikes,hitBatsmen,balks,wildPitches,pickoffsPitching,rbiPitching,gamesFinishedPitching,inheritedRunners,inheritedRunnersScored,catchersInterferencePitching,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances,id,name,teamName_y,teamCode,shortName,abbreviation,locationName,leagueId,leagueName,divisionId,divisionName,venueId,venueName,majormarket,awardId,awardName,awardDate,awardSeason,playerName,awardPlayerTeamId
0,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,680911,Owen Miller,1996-11-15,2021-05-23,Mequon,WI,USA,72,185,4,Second Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,670764,Taylor Walls,1996-07-10,2021-05-22,Americus,GA,USA,70,185,6,Shortstop,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,608422,Jose Godoy,1994-10-13,2021-05-21,Maracaibo,NaN,Venezuela,71,200,2,Catcher,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,642456,Luis Barrera,1995-11-15,2021-05-19,Tamboril,NaN,Dominican Republic,72,195,8,Outfielder,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [22]:
df4 = pd.merge(df3, nextDayPlayerEngagement, how='left', left_on=['playerId','dailyDataDate'], right_on=['playerId','dailyDataDate'])
df4

,playerId,playerName_x,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds,dailyDataDate,home,gamePk,gameDate,gameTimeUTC,teamId,teamName_x,playerName_y,jerseyNum,positionCode,positionName,positionType,battingOrder,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,winsPitching,lossesPitching,flyOutsPitching,airOutsPitching,groundOutsPitching,runsPitching,doublesPitching,triplesPitching,homeRunsPitching,strikeOutsPitching,baseOnBallsPitching,intentionalWalksPitching,hitsPitching,hitByPitchPitching,atBatsPitching,caughtStealingPitching,stolenBasesPitching,inningsPitched,saveOpportunities,earnedRuns,battersFaced,outsPitching,pitchesThrown,balls,strikes,hitBatsmen,balks,wildPitches,pickoffsPitching,rbiPitching,gamesFinishedPitching,inheritedRunners,inheritedRunnersScored,catchersInterferencePitching,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances,id,name,teamName_y,teamCode,shortName,abbreviation,locationName,leagueId,leagueName,divisionId,divisionName,venueId,venueName,majormarket,awardId,awardName,awardDate,awardSeason,playerName,awardPlayerTeamId,engagementMetricsDate,target1,target2,target3,target4
0,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,680911,Owen Miller,1996-11-15,2021-05-23,Mequon,WI,USA,72,185,4,Second Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,670764,Taylor Walls,1996-07-10,2021-05-22,Americus,GA,USA,70,185,6,Shortstop,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,608422,Jose Godoy,1994-10-13,2021-05-21,Maracaibo,NaN,Venezuela,71,200,2,Catcher,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,642456,Luis Barrera,1995-11-15,2021-05-19,Tamboril,NaN,Dominican Republic,72,195,8,Outfielder,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [23]:
df4.columns.values.tolist()

['playerId',
 'playerName_x',
 'DOB',
 'mlbDebutDate',
 'birthCity',
 'birthStateProvince',
 'birthCountry',
 'heightInches',
 'weight',
 'primaryPositionCode',
 'primaryPositionName',
 'playerForTestSetAndFuturePreds',
 'dailyDataDate',
 'home',
 'gamePk',
 'gameDate',
 'gameTimeUTC',
 'teamId',
 'teamName_x',
 'playerName_y',
 'jerseyNum',
 'positionCode',
 'positionName',
 'positionType',
 'battingOrder',
 'gamesPlayedBatting',
 'flyOuts',
 'groundOuts',
 'runsScored',
 'doubles',
 'triples',
 'homeRuns',
 'strikeOuts',
 'baseOnBalls',
 'intentionalWalks',
 'hits',
 'hitByPitch',
 'atBats',
 'caughtStealing',
 'stolenBases',
 'groundIntoDoublePlay',
 'groundIntoTriplePlay',
 'plateAppearances',
 'totalBases',
 'rbi',
 'leftOnBase',
 'sacBunts',
 'sacFlies',
 'catchersInterference',
 'pickoffs',
 'gamesPlayedPitching',
 'gamesStartedPitching',
 'completeGamesPitching',
 'shutoutsPitching',
 'winsPitching',
 'lossesPitching',
 'flyOutsPitching',
 'airOutsPitching',
 'groundOutsPitchin

In [24]:
df4a = df4.drop(['gamesPlayedPitching',
 'gamesStartedPitching',
 'completeGamesPitching',
 'shutoutsPitching',
 'winsPitching',
 'lossesPitching',
 'flyOutsPitching',
 'airOutsPitching',
 'groundOutsPitching',
 'runsPitching',
 'doublesPitching',
 'triplesPitching',
 'homeRunsPitching',
 'strikeOutsPitching',
 'baseOnBallsPitching',
 'intentionalWalksPitching',
 'hitsPitching',
 'hitByPitchPitching',
 'atBatsPitching',
 'caughtStealingPitching',
 'stolenBasesPitching',
 'inningsPitched',
 'saveOpportunities',
 'earnedRuns',
 'battersFaced',
 'outsPitching',
 'pitchesThrown',
 'balls',
 'strikes',
 'hitBatsmen',
 'balks',
 'wildPitches',
 'pickoffsPitching',
 'rbiPitching',
 'gamesFinishedPitching',
 'inheritedRunners',
 'inheritedRunnersScored',
 'catchersInterferencePitching',
 'sacBuntsPitching',
 'sacFliesPitching',
 'saves',
 'holds',
 'blownSaves'], axis=1)

In [25]:
#df4a = df4a[(df4a.gameDate >= '2021-02-28') & (df4a.gameDate <= '2021-10-31')]
df4a

,playerId,playerName_x,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds,dailyDataDate,home,gamePk,gameDate,gameTimeUTC,teamId,teamName_x,playerName_y,jerseyNum,positionCode,positionName,positionType,battingOrder,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,assists,putOuts,errors,chances,id,name,teamName_y,teamCode,shortName,abbreviation,locationName,leagueId,leagueName,divisionId,divisionName,venueId,venueName,majormarket,awardId,awardName,awardDate,awardSeason,playerName,awardPlayerTeamId,engagementMetricsDate,target1,target2,target3,target4
0,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,680911,Owen Miller,1996-11-15,2021-05-23,Mequon,WI,USA,72,185,4,Second Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,670764,Taylor Walls,1996-07-10,2021-05-22,Americus,GA,USA,70,185,6,Shortstop,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,608422,Jose Godoy,1994-10-13,2021-05-21,Maracaibo,NaN,Venezuela,71,200,2,Catcher,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,642456,Luis Barrera,1995-11-15,2021-05-19,Tamboril,NaN,Dominican Republic,72,195,8,Outfielder,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100193,667674,Jack Kruger,1994-10-26,NaN,Los Angeles,CA,USA,73,195,2,Catcher,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100194,672695,Geraldo Perdomo,1999-10-22,NaN,Santo Domingo,NaN,Dominican Republic,74,203,6,Shortstop,True,20210403.0,0.0,634623.0,2021-04-03,2021-04-04T00:40:00Z,109.0,Arizona Diamondbacks,Geraldo Perdomo,2,6.0,Shortstop,Infielder,201.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,109.0,Arizona Diamondbacks,D-backs,ari,Arizona,ARI,Phoenix,104.0,National League,203.0,National League West,15.0,Chase Field,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-04,1.264989,6.407224,1.281305,0.186374
100195,672695,Geraldo Perdomo,1999-10-22,NaN,Santo Domingo,NaN,Dominican Republi

In [26]:
df5 = pd.merge(df4a, playerTwitterFollowers, how='left', left_on=['playerId','dailyDataDate'], right_on=['playerId','dailyDataDate'])
df5

,playerId,playerName_x,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds,dailyDataDate,home,gamePk,gameDate,gameTimeUTC,teamId,teamName_x,playerName_y,jerseyNum,positionCode,positionName,positionType,battingOrder,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,assists,putOuts,errors,chances,id,name,teamName_y,teamCode,shortName,abbreviation,locationName,leagueId,leagueName,divisionId,divisionName,venueId,venueName,majormarket,awardId,awardName,awardDate,awardSeason,playerName_x,awardPlayerTeamId,engagementMetricsDate,target1,target2,target3,target4,date,playerName_y,accountName,twitterHandle,numberOfFollowers
0,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,680911,Owen Miller,1996-11-15,2021-05-23,Mequon,WI,USA,72,185,4,Second Base,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,670764,Taylor Walls,1996-07-10,2021-05-22,Americus,GA,USA,70,185,6,Shortstop,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,608422,Jose Godoy,1994-10-13,2021-05-21,Maracaibo,NaN,Venezuela,71,200,2,Catcher,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,642456,Luis Barrera,1995-11-15,2021-05-19,Tamboril,NaN,Dominican Republic,72,195,8,Outfielder,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100193,667674,Jack Kruger,1994-10-26,NaN,Los Angeles,CA,USA,73,195,2,Catcher,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
100194,672695,Geraldo Perdomo,1999-10-22,NaN,Santo Domingo,NaN,Dominican Republic,74,203,6,Shortstop,True,20210403.0,0.0,634623.0,2021-04-03,2021-04-04T00:40:00Z,109.0,Arizona Diamondbacks,Geraldo Perdomo,2,6.0,Shortstop,Infielder,201.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,109.0,Arizona Diamondbacks,D-backs,ari,Arizona,ARI,Phoenix,104.0,National Le

## Output Final Dataframe for Analysis

In [27]:
df5.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100198 entries, 0 to 100197
Data columns (total 84 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   playerId                        100198 non-null  int64         
 1   playerName_x                    100198 non-null  object        
 2   DOB                             100198 non-null  object        
 3   mlbDebutDate                    100189 non-null  object        
 4   birthCity                       100198 non-null  object        
 5   birthStateProvince              65586 non-null   object        
 6   birthCountry                    100198 non-null  object        
 7   heightInches                    100198 non-null  int16         
 8   weight                          100198 non-null  int32         
 9   primaryPositionCode             100198 non-null  object        
 10  primaryPositionName             100198 non-null  object 

In [28]:
#df5['gameDate'] = df5['gameDate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())
df5['gameDate'] = df5['gameDate'].apply(pd.to_datetime, format='%Y/%m/%d')

In [29]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100198 entries, 0 to 100197
Data columns (total 84 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   playerId                        100198 non-null  int64         
 1   playerName_x                    100198 non-null  object        
 2   DOB                             100198 non-null  object        
 3   mlbDebutDate                    100189 non-null  object        
 4   birthCity                       100198 non-null  object        
 5   birthStateProvince              65586 non-null   object        
 6   birthCountry                    100198 non-null  object        
 7   heightInches                    100198 non-null  int16         
 8   weight                          100198 non-null  int32         
 9   primaryPositionCode             100198 non-null  object        
 10  primaryPositionName             100198 non-null  object 

df5['pre_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2021-02-28') <= x <= pd.to_datetime('2021-03-30'):
                                       elif pd.to_datetime('2017-02-22') <= x <= pd.to_datetime('2017-04-01'):
                                       elif pd.to_datetime('2018-02-21') <= x <= pd.to_datetime('2018-03-27'):
                                       elif pd.to_datetime('2019-02-21') <= x <= pd.to_datetime('2019-03-26'):
                                       elif pd.to_datetime('2020-02-21') <= x <= pd.to_datetime('2020-07-22'):
                                       else 0).astype(int)

df5['all_star_game'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2021-07-13') == x:
                                          elif pd.to_datetime('2017-07-11') == x:
                                          elif pd.to_datetime('2018-07-17') == x:
                                          elif pd.to_datetime('2019-07-09') == x:
                                          else 0).astype(int)

df5['regular_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2021-04-01') <= x <= pd.to_datetime('2021-10-03'):
                                           elif pd.to_datetime('2017-04-02') <= x <= pd.to_datetime('2017-10-01'):
                                           elif pd.to_datetime('2018-03-29') <= x <= pd.to_datetime('2018-10-01'):
                                           elif pd.to_datetime('2019-03-20') <= x <= pd.to_datetime('2019-09-29'):
                                           elif pd.to_datetime('2020-07-23') <= x <= pd.to_datetime('2020-09-27'):
                                           else 0).astype(int)

df5['post_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2021-10-04') <= x <= pd.to_datetime('2021-10-31'):
                                        elif pd.to_datetime('2017-10-03') <= x <= pd.to_datetime('2017-11-01'):
                                        elif pd.to_datetime('2018-10-02') <= x <= pd.to_datetime('2018-10-28'):
                                        elif pd.to_datetime('2019-10-01') <= x <= pd.to_datetime('2019-10-30'):
                                        elif pd.to_datetime('2020-09-29') <= x <= pd.to_datetime('2020-10-28'):
                                        else 0).astype(int)

In [30]:
seasons

,seasonId,seasonStartDate,seasonEndDate,preSeasonStartDate,preSeasonEndDate,regularSeasonStartDate,regularSeasonEndDate,lastDate1stHalf,allStarDate,firstDate2ndHalf,postSeasonStartDate,postSeasonEndDate
0,2017,2017-04-02,2017-11-01,2017-02-22,2017-04-01,2017-04-02,2017-10-01,2017-07-09,2017-07-11,2017-07-14,2017-10-03,2017-11-01
1,2018,2018-03-29,2018-10-28,2018-02-21,2018-03-27,2018-03-29,2018-10-01,2018-07-15,2018-07-17,2018-07-19,2018-10-02,2018-10-28
2,2019,2019-03-20,2019-10-30,2019-02-21,2019-03-26,2019-03-20,2019-09-29,2019-07-07,2019-07-09,2019-07-11,2019-10-01,2019-10-30
3,2020,2020-07-23,2020-10-28,2020-02-21,2020-07-22,2020-07-23,2020-09-27,2020-08-25,NaN,2020-08-26,2020-09-29,2020-10-28
4,2021,2021-02-28,2021-10-31,2021-02-28,2021-03-30,2021-04-01,2021-10-03,2021-07-11,2021-07-13,2021-07-15,2021-10-04,2021-10-31


In [41]:
df5['pre_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2021-02-28') <= x <= pd.to_datetime('2021-03-30') or pd.to_datetime('2017-02-22') <= x <= pd.to_datetime('2017-04-01') or pd.to_datetime('2018-02-21') <= x <= pd.to_datetime('2018-03-27') or pd.to_datetime('2019-02-21') <= x <= pd.to_datetime('2019-03-26') or pd.to_datetime('2020-02-21') <= x <= pd.to_datetime('2020-07-22') else 0).astype(int)
df5['all_star_game'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2021-07-13') == x or pd.to_datetime('2017-07-11') == x or pd.to_datetime('2018-07-17') == x or pd.to_datetime('2019-07-09') == x else 0).astype(int)
df5['regular_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2021-04-01') <= x <= pd.to_datetime('2021-07-11') or pd.to_datetime('2021-07-11') <= x <= pd.to_datetime('2021-10-03') or pd.to_datetime('2017-04-02') <= x <= pd.to_datetime('2017-07-09') or pd.to_datetime('2017-07-14') <= x <= pd.to_datetime('2017-10-01') or pd.to_datetime('2018-03-29') <= x <= pd.to_datetime('2018-07-15') or pd.to_datetime('2018-07-19') <= x <= pd.to_datetime('2018-10-01') or pd.to_datetime('2019-03-20') <= x <= pd.to_datetime('2019-07-07') or pd.to_datetime('2019-07-11') <= x <= pd.to_datetime('2019-09-29') or pd.to_datetime('2020-07-23') <= x <= pd.to_datetime('2020-09-27') else 0).astype(int)
df5['post_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2021-10-04') <= x <= pd.to_datetime('2021-10-31') or pd.to_datetime('2017-10-03') <= x <= pd.to_datetime('2017-11-01') or pd.to_datetime('2018-10-02') <= x <= pd.to_datetime('2018-10-28') or pd.to_datetime('2019-10-01') <= x <= pd.to_datetime('2019-10-30') or pd.to_datetime('2020-09-29') <= x <= pd.to_datetime('2020-10-28') else 0).astype(int)

#df5['pre_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2017-02-22') <= x <= pd.to_datetime('2017-04-01') else 0).astype(int)
#df5['all_star_game'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2017-07-11') == x else 0).astype(int)
#df5['regular_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2017-04-02') <= x <= pd.to_datetime('2017-07-09') or pd.to_datetime('2017-07-14') <= x <= pd.to_datetime('2017-10-01') else 0).astype(int)
#df5['post_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2017-10-03') <= x <= pd.to_datetime('2017-11-01') else 0).astype(int)

#df5['pre_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2018-02-21') <= x <= pd.to_datetime('2018-03-27') else 0).astype(int)
#df5['all_star_game'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2018-07-17') == x else 0).astype(int)
#df5['regular_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2018-03-29') <= x <= pd.to_datetime('2018-07-15') or pd.to_datetime('2018-07-19') <= x <= pd.to_datetime('2018-10-01') else 0).astype(int)
#df5['post_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2018-10-02') <= x <= pd.to_datetime('2018-10-28') else 0).astype(int)

#df5['pre_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2019-02-21') <= x <= pd.to_datetime('2019-03-26') else 0).astype(int)
#df5['all_star_game'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2019-07-09') == x else 0).astype(int)
#df5['regular_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2019-03-20') <= x <= pd.to_datetime('2019-07-07') or pd.to_datetime('2019-07-11') <= x <= pd.to_datetime('2019-09-29') else 0).astype(int)
#df5['post_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2019-10-01') <= x <= pd.to_datetime('2019-10-30') else 0).astype(int)

#df5['pre_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2020-02-21') <= x <= pd.to_datetime('2020-07-22') else 0).astype(int)
#df5['all_star_game'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2019-07-09') == x else 0).astype(int)
#df5['regular_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2020-07-23') <= x <= pd.to_datetime('2020-09-27') else 0).astype(int)
#df5['post_season'] = df5.gameDate.apply(lambda x: 1 if pd.to_datetime('2020-09-29') <= x <= pd.to_datetime('2020-10-28') else 0).astype(int)

dates = pd.date_range(start='01/01/2017', end='01/01/2022')

df5 = pd.DataFrame({'gameDate': dates}).sample(10, ignore_index=True)

date_ranges_pre = list(map(pd.to_datetime, [('2021-02-28', '2021-03-30'),
                                        ('2017-02-22', '2017-04-01'),
                                        ('2018-02-21', '2018-03-27'),
                                        ('2019-02-21', '2019-03-26'),
                                        ('2020-02-21', '2020-07-22')]))

df5['pre_season'] = [1 if any([(start < x.to_pydatetime() < end)
                              for start, end in date_ranges_pre]) else 0
                     for x in df5['gameDate']]

date_ranges_reg = list(map(pd.to_datetime, [('2021-04-01', '2021-07-11'),
                                        ('2021-07-15', '2021-10-03'),
                                        ('2020-07-23', '2020-09-27'),
                                        ('2019-03-20', '2019-07-07'),
                                        ('2019-07-11', '2019-10-01')]))

df5['regular_season'] = [1 if any([(start < x.to_pydatetime() < end)
                              for start, end in date_ranges_reg]) else 0
                     for x in df5['gameDate']]

In [42]:
df = df5[['playerId',
'playerName_x',
'dailyDataDate',
'gamesPlayedBatting',
'flyOuts',
'groundOuts',
'runsScored',
'doubles',
'triples',
'homeRuns',
'strikeOuts',
'baseOnBalls',
'intentionalWalks',
'hits',
'hitByPitch',
'atBats',
'caughtStealing',
'stolenBases',
'groundIntoDoublePlay',
'groundIntoTriplePlay',
'plateAppearances',
'totalBases',
'rbi',
'leftOnBase',
'sacBunts',
'sacFlies',
'catchersInterference',
'pickoffs',
'assists',
'putOuts',
'errors',
'chances',
'name',
'leagueName',
'divisionName',
'majormarket',
'awardId',
'awardName',
'engagementMetricsDate',
'target1',
'target2',
'target3',
'target4',
'numberOfFollowers',
'pre_season',        
'all_star_game',
'regular_season',
'post_season'
         ]]

In [43]:
#df = df[df['dailyDataDate','leagueName'].notna()]
df.dropna(subset = ['dailyDataDate'])
#df.dropna(subset = ['majormarket'])
df = df[df['leagueName'].notnull()]

In [44]:
df.columns.values.tolist()

['playerId',
 'playerName_x',
 'playerName_x',
 'dailyDataDate',
 'gamesPlayedBatting',
 'flyOuts',
 'groundOuts',
 'runsScored',
 'doubles',
 'triples',
 'homeRuns',
 'strikeOuts',
 'baseOnBalls',
 'intentionalWalks',
 'hits',
 'hitByPitch',
 'atBats',
 'caughtStealing',
 'stolenBases',
 'groundIntoDoublePlay',
 'groundIntoTriplePlay',
 'plateAppearances',
 'totalBases',
 'rbi',
 'leftOnBase',
 'sacBunts',
 'sacFlies',
 'catchersInterference',
 'pickoffs',
 'assists',
 'putOuts',
 'errors',
 'chances',
 'name',
 'leagueName',
 'divisionName',
 'majormarket',
 'awardId',
 'awardName',
 'engagementMetricsDate',
 'target1',
 'target2',
 'target3',
 'target4',
 'numberOfFollowers',
 'pre_season',
 'all_star_game',
 'regular_season',
 'post_season']

In [45]:
df.update(df[['gamesPlayedBatting',
 'flyOuts',
 'groundOuts',
 'runsScored',
 'doubles',
 'triples',
 'homeRuns',
 'strikeOuts',
 'baseOnBalls',
 'intentionalWalks',
 'hits',
 'hitByPitch',
 'atBats',
 'caughtStealing',
 'stolenBases',
 'groundIntoDoublePlay',
 'groundIntoTriplePlay',
 'plateAppearances',
 'totalBases',
 'rbi',
 'leftOnBase',
 'sacBunts',
 'sacFlies',
 'catchersInterference',
 'pickoffs',
 'assists',
 'putOuts',
 'errors',
 'chances',]].fillna(0))

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100099 entries, 17 to 100197
Data columns (total 49 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   playerId               100099 non-null  int64  
 1   playerName_x           100099 non-null  object 
 2   playerName_x           291 non-null     object 
 3   dailyDataDate          100099 non-null  float64
 4   gamesPlayedBatting     100099 non-null  float32
 5   flyOuts                100099 non-null  float32
 6   groundOuts             100099 non-null  float32
 7   runsScored             100099 non-null  float32
 8   doubles                100099 non-null  float32
 9   triples                100099 non-null  float32
 10  homeRuns               100099 non-null  float32
 11  strikeOuts             100099 non-null  float32
 12  baseOnBalls            100099 non-null  float32
 13  intentionalWalks       100099 non-null  float32
 14  hits                   100099 non-n

In [47]:
df.to_pickle("./MLBEngagementData2021NonPitchers.pkl")

In [48]:
df

,playerId,playerName_x,playerName_x,dailyDataDate,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,assists,putOuts,errors,chances,name,leagueName,divisionName,majormarket,awardId,awardName,engagementMetricsDate,target1,target2,target3,target4,numberOfFollowers,pre_season,all_star_game,regular_season,post_season
17,666163,Ben Rortvedt,NaN,20210430.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,7.0,Minnesota Twins,American League,American League Central,0.0,NaN,NaN,2021-05-01,1.216004,5.488800,0.657555,3.232047,NaN,0,0,1,0
18,613564,Jason Vosler,NaN,20210424.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,San Francisco Giants,National League,National League West,0.0,NaN,NaN,2021-04-25,1.479820,9.215017,5.959181,3.869677,NaN,0,0,1,0
19,613564,Jason Vosler,NaN,20210425.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,San Francisco Giants,National League,National League West,0.0,NaN,NaN,2021-04-26,2.391698,2.677821,2.006855,3.015009,NaN,0,0,1,0
20,613564,Jason Vosler,NaN,20210426.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,San Francisco Giants,National League,National League West,0.0,NaN,NaN,2021-04-27,4.078845,4.898914,0.206044,2.071601,NaN,0,0,1,0
21,613564,Jason Vosler,NaN,20210427.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,San Francisco Giants,National League,National League West,0.0,NaN,NaN,2021-04-28,3.591404,1.822437,0.091351,1.104625,NaN,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100192,666801,Rodolfo Castro,NaN,20210421.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,Pittsburgh Pirates,National League,National League Central,0.0,NaN,NaN,2021-04-22,0.014543,3.599019,0.018198,0.755064,NaN,0,0,1,0
100194,672695,Geraldo Perdomo,NaN,20210403.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Arizona Diamondbacks,National League,National League West,0.0,NaN,NaN,2021-04-04,1.264989,6.407224,1.281305,0.186374,NaN,0,0,1,0
100195,672695,Geraldo Perdomo,NaN,20210404.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,4.0,Arizona Diamondbacks,National League,National League West,0.0,NaN,NaN,2021-04-05,0.464332,2.019461,0.141061,0.129661,NaN,0,0,1,0
100196,672695,Geraldo Perdomo,NaN,20210406.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,6.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,5.0,Arizona Diamondbacks,National League,National League West,0.0,NaN,NaN,2021-04-07,0.551493,4.623298,0.361535,0.463519,NaN,0,0,1,0
